# Renault car sequencing I -- Exploration

Version 14/05/22

Herkunft von Problem und Daten:
https://www.roadef.org/challenge/2005/en/


Wir importieren ein paar Pakete. (Nicht alle brauchen wir hier.) 
<ul>
    <li><code>numpy</code>: wissenschaftliche Bibliothek, im wesentlichen für die array-Klasse</li>
    <li><code>matplotlib</code>: Visualisierungsfunktionen</li>
    <li><code>math</code>: mathematische Funktionen</li>
</ul>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math

plt.style.use("seaborn")  ## plots sehen so ein bisschen schöner aus

Als nächstes werden die Instanzdaten gelesen und eine Bibliothek (<code>rnlt</code>, kurz für *Renault*) mit Hilfscode (data handling und Visualisierung) importiert. Weiterhin werden die Instanzdaten und <code>rnlt</code> automatisch heruntergeladen -- dies funktioniert so wie es dort steht unter Google Colab und unter Linux (evtl auch iOs). Wenn Sie den Code auf einem Windows-Rechner lokal ausführen wollen, dann laden Sie die Instanzdaten unter (https://www.roadef.org/challenge/2005/files/Instances_set_A.zip)(https://www.roadef.org/challenge/2005/files/Instances_set_A.zip) herunter, dann entpacken Sie die Daten und speichern Sie die Daten und <code>rnlt.py</code> im gleichen Ordner wie dieses juypter-Notebook. Weiterhin müssen Sie die ersten drei Zeilen auskommentieren.

In [2]:
!wget -q https://www.roadef.org/challenge/2005/files/Instances_set_A.zip
!unzip -q Instances_set_A.zip
!wget -q https://raw.githubusercontent.com/henningbruhn/mobp/main/sequencing_aufgabe/rnlt.py

# Pfad zu den gespeicherten Instanzen -- ersetzen Sie das evtl durch den entsprechenden Pfad bei Ihnen
#path="RENAULT/Instances_set_A/"   
path="Instances"

import rnlt

## Methoden zum Einlesen aller Instanzen

Die Daten bestehen aus eine Reihe von Instanzen. Jede Instanz umfasst die Produktion an einem Tag in einer Fertigungsstätte. Renault verfügt über mehrere Fertigungsstätten. Entsprechend können sich die Instanzen unterscheiden.
Alle Instanzen unter path werden mit der Hilfsfunktion <code>read_in_all_instances</code> aus <code>rnlt</code> in das dictionary <code>data_dict</code> eingelesen. 

In [3]:
data_dict=rnlt.read_in_all_instances(path)

Folgende Instanzen wurden eingelesen: 
  039_38_4_EP_RAF_ch1
  024_38_3_EP_ENP_RAF
  048_39_1_EP_ENP_RAF
  025_38_1_EP_ENP_RAF
  064_38_2_EP_RAF_ENP_ch1
  064_38_2_RAF_EP_ENP_ch2
  022_3_4_EP_RAF_ENP
  048_39_1_EP_RAF_ENP
  024_38_5_EP_RAF_ENP
  039_38_4_RAF_EP_ch1
  024_38_3_EP_RAF_ENP
  025_38_1_EP_RAF_ENP
  064_38_2_EP_RAF_ENP_ch2
  064_38_2_RAF_EP_ENP_ch1
  022_3_4_RAF_EP_ENP
  024_38_5_EP_ENP_RAF


Wir gucken uns nun eine Instanz an. Jede Instanz ist wiederum ein dictionary und hält verschiedene Daten bereit. Mit <code>dictionary.keys()</code> erhalten wir eine Liste der im dictionary abgespeicherten Schlüsselwörter.

In [4]:
sample_instance=data_dict['064_38_2_EP_RAF_ENP_ch1']
sample_instance.keys()

dict_keys(['ratios', 'weights', 'current_day', 'renault_schedule', 'paint_batch_limit', 'name'])

Gucken wir uns die verschiedenen Einträge an. <code>'name'</code> ist, wenig überraschend, der Name der Instanz.

In [5]:
sample_instance['name']

'064_38_2_EP_RAF_ENP_ch1'

Der Eintrag <code>'current_day'</code> enthält die Fahrzeuge, die gereiht werden müssen.

In [6]:
sample_instance['current_day'][:10]  ## nur die ersten 10 Einträge

[64033840771,
 64033750048,
 64033820923,
 64033820874,
 64033810649,
 64033820888,
 64033820985,
 64033806803,
 64033840889,
 64033810753]

Was wir sehen: die Id-Nummern der Fahrzeuge. In der Liste sind aber tatsächlich die Fahrzeuge gespeichert, und zwar als Objekte. Betrachten wir ein Fahrzeug.

In [7]:
car=sample_instance['current_day'][42]
print("Fahrzeug Id-Nr {} hat Farbe {}".format(car.ident,car.colour))
print("Es sollen folgende Optionen verbaut werden: {}".format(car.options))

Fahrzeug Id-Nr 64033840854 hat Farbe 7
Es sollen folgende Optionen verbaut werden: ['HPRC5', 'LPRC2']


Dh jeder Eintrag <code>car</code> in der Liste <code>sample_instance['current_day']</code> hat die Eigenschaften
<ul>
    <li><code>car.ident</code>: die Id-Nr</li>
    <li><code>car.colour</code>: die Farbe</li>
    <li><code>car.options</code>: die zu verbauenden Optionen</li>
</ul>

Mit <code>print</code> können diese Eigenschaft direkt ausgegeben werden.

In [8]:
print(sample_instance['current_day'][6])

id: 64033820985, Farbe: 1, Optionen: ['HPRC7', 'LPRC2']


Weiterhin enthält die Instanz Information zu den $p/q$-Bedinungen, abgespeichert als dictionary. Die Einträge sind jeweils als <code>(p,q)</code> abgelegt.

In [9]:
sample_instance['ratios']

{'HPRC1': (1, 8),
 'HPRC2': (1, 2),
 'HPRC3': (1, 7),
 'HPRC4': (1, 15),
 'HPRC5': (2, 3),
 'HPRC6': (1, 30),
 'HPRC7': (4, 5),
 'LPRC1': (1, 6),
 'LPRC2': (1, 3)}

Renault hat jeweils bereits eine Reihung algorithmisch ermittelt. Zum Vergleich können wir sie heranziehen. Wir wollen natürlich besser sein.

In [10]:
sample_instance['renault_schedule'][:10]  ## wir zeigen nur die ersten 10 Einträge an

[64033840771,
 64033750048,
 64033820923,
 64033820874,
 64033810649,
 64033820888,
 64033820985,
 64033806803,
 64033840889,
 64033810753]

Es folgen zwei weitere Parameter: das paint_batch_limit und die Gewichtung der Ziele

In [11]:
sample_instance['paint_batch_limit']

15

In [12]:
sample_instance['weights']

{'paint': 1000, 'high': 1000000, 'low': 1}

Dies ist so zu lesen: Größte Priorität bei der Optimierung wird den high-priority Optionen zugewiesen, dann folgt die Zahl der Farbwechsel und schließlich die low-priority Optionen. 

Um das Ziel der Optimierung besser für den/die Anwender/in zu verdeutlichen, gibt's auch noch eine Methode:

In [13]:
rnlt.prio_string(sample_instance)

'high >> paint >> low'

## 1. Aufgabe: Exploration der Instanzen

Bevor der erste Algorithmus formuliert wird, sollten die Daten gesichtet werden. Manchmal ergibt sich die eine oder andere Überraschung: ein Aspekt des Problems ist anders als vom Anwender dargestellt, andere Aspekte erweisen sich als unwichtig, da sie kaum vorkommen usw. Wichtig ist es auch, eine Idee vom Umfang des Problems zu erhalten. Wie groß sind überhaupt die Instanzen?

Überlegen Sie sich, welche Daten Sie über die Instanzen erheben wollen. Betrachten Sie einzelne Instanzen und aber auch die Gesamtheit der Instanzen. Welche Daten können Ihnen Einblicke in das Problem verschaffen? Beispielhaft listen wir als erstes die Instanzgrößen auf, dh, wie viele Fahrzeuge jeweils produziert werden sollen:

In [14]:
sizes=[len(instance['current_day']) for instance in data_dict.values()]
sorted(sizes)

[335,
 335,
 485,
 485,
 600,
 600,
 875,
 875,
 954,
 954,
 1004,
 1004,
 1260,
 1260,
 1315,
 1315]

Und jetzt Sie! Lassen Sie sich das <code>paint_batch_limit</code> über alle Instanzen ausgeben. Gucken Sie welche Optimierungsziele vorgeschrieben werden. Nehmen Sie sich eine, zwei Instanzen und zählen Sie wie oft jede Farbe und jedes Extra vorkommt. Gibt es weitere Kennzahlen, die nützlich wären?

## Lösung

Als erstes geben wir die <code>paint_batch_limit</code> aus.

In [15]:
paint_batch_limits=[instance['paint_batch_limit'] for instance in data_dict.values()]
sorted(paint_batch_limits)

[10, 10, 10, 10, 10, 10, 10, 10, 15, 15, 15, 15, 15, 20, 450, 450]

Als nächstes schauen wir, welche Optimierungsziele vorgeschrieben sind:

In [16]:
count_prios={}
for instance in data_dict.values():
    prio=rnlt.prio_string(instance)
    count_prios[prio]=count_prios.get(prio,0)+1
count_prios

{'high >> paint >> low': 8,
 'high >> low >> paint': 4,
 'paint >> high >> low': 4}

Nun betrachten wir die Farben in einer Instanz.

In [17]:
def count_colours_and_options(instance):
    colour_count={}
    option_count={}
    for car in instance['current_day']:
        colour_count[car.colour]=colour_count.get(car.colour,0)+1
        for option in car.options:
            option_count[option]=option_count.get(option,0)+1
    return colour_count,option_count

def print_colour_summary(instance):
    colour_count,option_count=count_colours_and_options(instance)
    print("**Farben**")
    for key in colour_count.keys():
        print("Farbe {}: {}-mal".format(key,colour_count[key]))

print_colour_summary(sample_instance)

**Farben**
Farbe 5: 177-mal
Farbe 1: 185-mal
Farbe 6: 1-mal
Farbe 7: 148-mal
Farbe 8: 77-mal
Farbe 9: 67-mal
Farbe 10: 37-mal
Farbe 11: 19-mal
Farbe 2: 28-mal
Farbe 12: 45-mal
Farbe 13: 29-mal
Farbe 14: 10-mal
Farbe 3: 38-mal
Farbe 4: 14-mal


Für die Extras berechnen wir gleich ein wenig mehr. Wir schreiben gleich eine Funktion, die alle wesentlichen Daten über die Instanz ausgibt. Dazu schätzen wir die Mindestanzahl der Farbwechsel ab. Auch interessant: Wie viel *Luft* ist bei den jeweiligen Extras, dh, wenn die nachgefragten Extras gleichmäßig über die Reihung verteilt werden, wie viele Extras könnten noch zusätzlich verbaut werden, bevor die erste p/q-Verletzung eintritt?

In [18]:
from operator import itemgetter
def lower_bound_colour_changes(colour_count,instance):
    colour_changes=0
    paint_batch_limit=instance['paint_batch_limit']
    for colour in colour_count.keys():
        colour_changes+=(colour_count[colour]//paint_batch_limit) +1 ## technically, would need to subtract 1 from colour_count
    return colour_changes

def lower_bound_option_surplus(option_count,instance):
    N=len(instance['current_day'])
    result=[]
    for option in option_count.keys():
        p,q=instance['ratios'][option]
        #surplus=math.ceil(max(option_count[option]-N*p/q,0))
        surplus=math.ceil(N*p/q-option_count[option])
        option_result=(option,p,q,option_count[option],surplus)
        result.append(option_result)
    return sorted(result,key=itemgetter(0))  # cosmetics

def print_stats(instance):
    print("Instanz {}".format(instance['name']))
    print("Priorität: "+rnlt.prio_string(instance))
    N=len(instance['current_day'])
    print("Fahrzeuge: {}".format(N))
    print("paint_batch_limit: {}".format(instance['paint_batch_limit']))
    colour_count,option_count=count_colours_and_options(instance)
    colour_changes=lower_bound_colour_changes(colour_count,instance)
    print("Farben | Min-Farbwechsel: {} | {}".format(len(colour_count.keys()),colour_changes))
    options=lower_bound_option_surplus(option_count,instance)
    for item in options:
        print("Option | p/q | Anzahl | balance: {} | {}/{} | {} | {}".format(*item))
    
print_stats(sample_instance)

Instanz 064_38_2_EP_RAF_ENP_ch1
Priorität: high >> paint >> low
Fahrzeuge: 875
paint_batch_limit: 15
Farben | Min-Farbwechsel: 14 | 65
Option | p/q | Anzahl | balance: HPRC1 | 1/8 | 56 | 54
Option | p/q | Anzahl | balance: HPRC2 | 1/2 | 181 | 257
Option | p/q | Anzahl | balance: HPRC3 | 1/7 | 56 | 69
Option | p/q | Anzahl | balance: HPRC4 | 1/15 | 14 | 45
Option | p/q | Anzahl | balance: HPRC5 | 2/3 | 375 | 209
Option | p/q | Anzahl | balance: HPRC6 | 1/30 | 16 | 14
Option | p/q | Anzahl | balance: HPRC7 | 4/5 | 500 | 200
Option | p/q | Anzahl | balance: LPRC1 | 1/6 | 59 | 87
Option | p/q | Anzahl | balance: LPRC2 | 1/3 | 534 | -242


Möglicherweise gibt es nur ganz wenige unterschiedliche Typen von Extras/Farb-Kombinationen. Zählen wir mal.

In [19]:
def car_type(car):
    T=sorted(car.options)
    T.append(car.colour)
    return tuple(T)

def unique_types(instance):
    types=[]
    for car in instance['current_day']:
        t=car_type(car)
        if not t in types:
            types.append(t)
    return types

U=unique_types(sample_instance)
len(U)

133

...leider nicht der Fall.